## Imports and Installs

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
# Install yfinance API
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import pandas as pd
import datetime
import yfinance as yf
import numpy as np
from pandas_datareader import data as pdr

## Load a company's data

In [ ]:
# Sample company - WMT
company = 'WMT'
df_tweets = pd.read_csv('/content/drive/My Drive/Data from 2010/stocktwits_' + company + '.csv')
df_tweets.tail()

Split datetime into date and time


In [ ]:
df_tweets['datetime'] = df_tweets['datetime'].astype('datetime64[ns]')
df_tweets['Date'] = [d.date() for d in df_tweets['datetime']]
df_tweets['Time'] = [d.time() for d in df_tweets['datetime']]
df_tweets

In [ ]:
df_tweets['Day_of_week'] = df_tweets.datetime.dt.day_name()

# Get End date
if df_tweets.iloc[0][7] == 'Sunday':
  endDate = df_tweets.iloc[0][5] + datetime.timedelta(days = 2)
elif df_tweets.iloc[0][7] == 'Saturday':
  endDate = df_tweets.iloc[0][5] + datetime.timedelta(days = 3)
else:
  endDate = df_tweets.iloc[0][5] + datetime.timedelta(days = 1)

# Get start date
if df_tweets.iloc[-1][7] == 'Sunday':
  startDate = df_tweets.iloc[-1][5] - datetime.timedelta(days = 2)
elif df_tweets.iloc[-1][7] == 'Saturday':
  startDate = df_tweets.iloc[-1][5] - datetime.timedelta(days = 1)
else:
  startDate = df_tweets.iloc[-1][5]

print(endDate) 
print(startDate) 

## Download a company's yahoo finance data

In [ ]:
yahoo_data = yf.download(company, start = startDate, end = endDate)
yahoo_data.reset_index(level=0, inplace=True)
yahoo_data.columns

In [ ]:
yahoo_data

## Fill missing data

Fill missing dates (weekends and public holidays)


In [ ]:
start_date = startDate
end_date = endDate

df_prices = pd.DataFrame(yahoo_data)
all_days = pd.date_range(start=start_date, end=end_date, freq='D')
df_prices.info

df_prices['Date'] =  pd.to_datetime(df_prices['Date'], format='%Y/%m/%d')
df_prices = df_prices.sort_values(by=['Date'], ascending=[True])
df_prices.set_index('Date', inplace=True)
df_prices = df_prices.reindex(all_days).reset_index().rename(columns={"index":"Date"})
df_prices

Remove last row with NaN


In [ ]:
if np.isnan(df_prices.iloc[-1][1]):
  df_prices.drop(df_prices.tail(1).index,inplace=True)
df_prices

Fill missing values for missing dates by calculating mean of top and bottom row


In [ ]:
for i in range(len(df_prices)):
  if np.isnan(df_prices.iloc[i][1]): # Continue if open price is NaN
    counter = 1 
    while counter != 0:
      if ~np.isnan(df_prices.iloc[i+counter][1]): # Find the next non-NaN valued row using counter
        for j in range(1, 7):
          new = (df_prices.iloc[i - 1][j] + df_prices.iloc[i + counter][j]) / 2 # Calculate mean
          df_prices.at[i, df_prices.columns[j]] = new
        break
      counter += 1
df_prices.tail(15)

## Calculate Price Change

Calculate Price Change using Close price and Open Price of Same day


In [ ]:
# price_change = []
# for i in range(len(df_prices)):
#   if df_prices.iloc[i][4] > df_prices.iloc[i][1]:
#     price_change.append(1)
#   else:
#     price_change.append(0)
# df_prices['PriceChange'] = price_change
# df_prices.tail(15)

Calculate Price Change using Close price of Previous day with Close price of today


In [ ]:
price_change = []
for i in range(len(df_prices)):
  if df_prices.iloc[i-1][4] < df_prices.iloc[i][4]:
    price_change.append(1)
  else:
    price_change.append(0)
df_prices['PriceChange'] = price_change
df_prices.head(15)

## Assign polarity / label to tweets

Function to return Price Change value for a particular date


In [ ]:
def get_polarity (date):
  rslt_df = df_prices[df_prices['Date'] == date] 
  return rslt_df['PriceChange'].values[0]

Assign polarity/label for each row using date


In [ ]:
for i in range(len(df_tweets)):
  # print(i)
  polarity = get_polarity(df_tweets.iloc[i][5].strftime("%Y-%m-%d"))
  df_tweets.at[i, 'polarity'] = polarity
df_tweets[:100]

Check null or NaN values


In [ ]:
print(df_prices.isna().values.any()) #False
print(df_prices.notnull().values.all()) #True
print(df_prices.isnull().values.any()) #False

Convert float values of polarity/label to integer 


In [ ]:
df_tweets["polarity"] = df_tweets["polarity"].astype(float).astype(int)

## Save labelled company file

In [ ]:
df_tweets.to_csv('labelled_WMT.csv')
!cp labelled_WMT.csv "/content/drive/My Drive/Labelled (Binary - previous day open close)"

In [ ]:
df_prices.info()
df_tweets.info()